In [11]:
pip install sqlalchemy psycopg2-binary pandas python-dotenv

183.62s - pydevd: Sending message related to process being replaced timed-out after 5 seconds



[notice] A new release of pip is available: 25.0 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import random
from datetime import datetime, timedelta
import sys
import os  
from dotenv import load_dotenv  

load_dotenv()  

try:

    CONNECTION_STRING = os.getenv('POSTGRES_CONNECTION_STRING')
    if not CONNECTION_STRING:
        raise ValueError("Environment variable 'POSTGRES_CONNECTION_STRING' not found. \nPlease ensure it is set in your .env file.")
except Exception as e:
    print(e, file=sys.stderr)
    sys.exit(" Could not load environment variable.")

try:
    engine = create_engine(CONNECTION_STRING)
  
    with engine.connect() as conn:
        print(" Connected to PostgreSQL Database!")
except Exception as e:
    print(f" Failed to connect to PostgreSQL:", file=sys.stderr)
    print(e, file=sys.stderr)
    sys.exit()


random.seed(42)

INTENTS = [
    "Cancel Trip", "Cancellation Policy", "Carry On Luggage Faq", "Change Flight",
    "Check In Luggage Faq", "Complaints", "Damaged Bag", "Discounts", "Fare Check",
    "Flight Status", "Flights Info", "Insurance", "Medical Policy", "Missing Bag",
    "Pet Travel", "Prohibited Items Faq", "Seat Availability", "Sports Music Gear"
]

AIRPORTS = ["JFK", "LAX", "SFO", "DEL", "BOM", "DXB", "LHR", "CDG", "SIN", "HND"]
CITIES = ["New York", "Los Angeles", "San Francisco", "Delhi", "Mumbai", "Dubai", "London", "Paris", "Singapore", "Tokyo"]
NAMES = ["John", "Aisha", "Rahul", "Maria", "Chen", "Sara", "David", "Priya"]
FLIGHT_PREFIX = ["AI", "BA", "EK", "UA", "DL", "LH", "SQ"]
AMOUNTS = ["50", "100", "200", "300"] 
ITEMS = ["guitar","surfboard","violin","laptop","power bank"]

POLITE_PREFIXES = ["Hi,", "Hello,", "Hey,", "Good morning,", "Good evening,"]
POLITE_SUFFIXES = ["please", "thanks", "thank you", "kindly"]

TEMPLATES = {
    "Cancel Trip": [
        "I want to cancel my flight to {city}.",
        "Please cancel my booking for flight {flight}.",
        "Can you cancel my ticket and refund me?",
        "How do I cancel my trip scheduled on {date}?",
        "I need to cancel my reservation for {city}."
    ],
    "Cancellation Policy": [
        "What is your cancellation policy?",
        "Do I get a refund if I cancel within 24 hours?",
        "How does the cancellation fee work?",
        "Tell me about the cancellation rules for tickets.",
        "Is there a penalty if I cancel my booking?"
    ],
    "Carry On Luggage Faq": [
        "What is the carry-on baggage allowance?",
        "Can I carry a backpack and a laptop?",
        "What can I bring in my carry-on?",
        "How many carry-on bags are allowed?",
        "Is a guitar allowed as carry-on?"
    ],
    "Change Flight": [
        "I need to change my flight date to {date}.",
        "Can you help me reschedule flight {flight}?",
        "How can I change my booked flight to {city}?",
        "Change my flight to an earlier time please.",
        "I want to move my booking to another day."
    ],
    "Check In Luggage Faq": [
        "What is the checked baggage allowance for economy?",
        "Can I check two bags?",
        "How many kilograms can I check in?",
        "Is overweight baggage charged and how much?",
        "What are the check-in baggage rules?"
    ],
    "Complaints": [
        "I want to make a complaint about my last flight.",
        "I had a terrible experience and want to complain.",
        "Who can I contact to file a complaint?",
        "I want to report bad service on flight {flight}.",
        "I am unhappy with the customer service I received."
    ],
    "Damaged Bag": [
        "My luggage arrived damaged. What do I do?",
        "I received a damaged bag at {airport}.",
        "How do I claim for a damaged bag?",
        "There is a tear in my suitcase after the flight.",
        "Who handles baggage damage claims?"
    ],
    "Discounts": [
        "Are there any discounts for students?",
        "Do you have promo codes for flights?",
        "Is there a discount for early bookings?",
        "How can I apply a coupon to my booking?",
        "Any special fares or offers available now?"
    ],
    "Fare Check": [
        "What is the fare for a flight from {city} to {city2} on {date}?",
        "How much is the ticket to {city} next week?",
        "Check price for flight {flight}.",
        "Lowest fare from {city} to {city2}.",
        "What's the current price for a one-way ticket to {city}?"
    ],
    "Flight Status": [
        "What's the status of flight {flight}?",
        "Is my flight on time?",
        "Has flight {flight} landed?",
        "When is flight {flight} departing?",
        "Check flight status for {flight}."
    ],
    "Flights Info": [
        "What are the baggage rules for international flights?",
        "How long is the layover in {city}?",
        "What is the boarding time before departure?",
        "Tell me about meals on this flight.",
        "Are Wi-Fi services available on board?"
    ],
    "Insurance": [
        "Do you offer travel insurance?",
        "How do I buy insurance for my ticket?",
        "What does the travel insurance cover?",
        "Is medical insurance included with the booking?",
        "What's the cost of travel insurance?"
    ],
    "Medical Policy": [
        "What is the medical policy for passengers with special needs?",
        "Can I travel with medical oxygen?",
        "What documents are required for medical travel?",
        "Are there rules for traveling while pregnant?",
        "Is assistance available for disabled passengers?"
    ],
    "Missing Bag": [
        "My bag did not arrive at {airport}.",
        "I can't find my checked-in luggage.",
        "File a missing baggage report for flight {flight}.",
        "Where is my suitcase? It didn't come out on carousel.",
        "How do I report a lost bag?"
    ],
    "Pet Travel": [
        "Can I bring my dog on the flight?",
        "What is the pet travel policy?",
        "Do pets travel in cargo or cabin?",
        "Is there a fee for traveling with a pet?",
        "What documents are required for pet travel?"
    ],
    "Prohibited Items Faq": [
        "Is a lighter allowed in checked baggage?",
        "What items are prohibited on flights?",
        "Can I carry a power bank in my bag?",
        "Are sharp objects allowed in checked luggage?",
        "Can I bring a vape on board?"
    ],
    "Seat Availability": [
        "Are there seats available on flight {flight}?",
        "How do I check seat availability?",
        "Is there a window seat left for my booking?",
        "Can I reserve a seat on the flight?",
        "What seats are available in business class?"
    ],
    "Sports Music Gear": [
        "Can I take my guitar on board?",
        "What's the policy for carrying sports equipment?",
        "Is my surfboard allowed on the flight?",
        "Can I bring a violin as carry-on?",
        "Is there any extra charge for sports gear?"
    ]
}


def random_date_within(days=60):
    return (datetime.today() + timedelta(days=random.randint(1, days))).strftime("%Y-%m-%d")

def random_flight_number():
    return random.choice(FLIGHT_PREFIX) + str(random.randint(100, 999))

def random_airport():
    return random.choice(AIRPORTS)

def random_city(exclude=None):
    choices = [c for c in CITIES if c != exclude] if exclude else CITIES
    return random.choice(choices)

def maybe_polite(text):
    if random.random() < 0.25: text = random.choice(POLITE_PREFIXES) + " " + text
    if random.random() < 0.25: text = text + " " + random.choice(POLITE_SUFFIXES)
    return text

def inject_typo(text, prob=0.08):
    if random.random() > prob: return text
    words = text.split()
    if not words: return text
    idx = random.randint(0, len(words)-1)
    w = words[idx]
    if len(w) > 1:
        pos = random.randint(0, len(w)-1)
        if random.random() < 0.33:
            w = w[:pos] + w[pos+1:]
        elif random.random() < 0.66 and pos < len(w)-1:
            w = w[:pos] + w[pos+1] + w[pos] + w[pos+2:]
        else:
            w = w[:pos] + random.choice('abcdefghijklmnopqrstuvwxyz') + w[pos+1:]
        words[idx] = w
    return " ".join(words)

def paraphrase_synonyms(text):
    replacements = {
        "cancel": ["cancel", "terminate", "call off", "call it off"],
        "change": ["change", "reschedule", "modify", "move"],
        "flight": ["flight", "trip", "booking", "ticket"],
        "bag": ["bag", "luggage", "baggage"],
        "check-in": ["check-in", "check in", "checking in"],
        "status": ["status", "state", "timing"],
        "allowed": ["allowed", "permitted", "okay to bring"],
        "fee": ["fee", "charge", "penalty"]
    }
    lower = text.lower()
    for k,v in replacements.items():
        idx = lower.find(k)
        if idx != -1:
            choice = random.choice(v)
            if text[idx].isupper(): choice = choice.capitalize()
            text = text[:idx] + choice + text[idx+len(k):]
    return text

def generate_for_intent(intent, n=60):
    templates = TEMPLATES.get(intent, [""])
    samples = []
    for _ in range(n):
        t = random.choice(templates)
        filled = t.format(
            city=random_city(),
            city2=random_city(exclude=random_city()),
            airport=random_airport(),
            flight=random_flight_number(),
            date=random_date_within(),
            name=random.choice(NAMES),
            amount=random.choice(AMOUNTS),
            item=random.choice(ITEMS)
        )
        if random.random() < 0.4: filled = paraphrase_synonyms(filled)
        filled = maybe_polite(filled)
        filled = inject_typo(filled, prob=0.08)
        if random.random() < 0.1: filled = filled.upper()
        elif random.random() < 0.1: filled = filled.capitalize()
        samples.append(filled.strip())
    return samples

def generate_dataset(samples_per_intent=60):
    rows = []
    for intent in INTENTS:
        samples = generate_for_intent(intent, n=samples_per_intent)
        for s in samples:
     
            rows.append({"utterance": s, "intent": intent})
    df = pd.DataFrame(rows)
    return df.sample(frac=1, random_state=42).reset_index(drop=True)


print("Generating synthetic dataset...")
df = generate_dataset(samples_per_intent=60)
print(f" Generated {len(df)} total samples.")


try:
    df.to_sql("intents_full", engine, if_exists="replace", index=False)
    print(f" Uploaded {len(df)} rows into 'intents_full' table successfully!")
except Exception as e:
    print(f" Failed to upload full dataset:", file=sys.stderr)
    print(e, file=sys.stderr)



train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

print(f"\n Train set: {len(train_df)} rows |  Test set: {len(test_df)} rows")

try:
    train_df.to_sql("train_set", engine, if_exists="replace", index=False)
    test_df.to_sql("test_set", engine, if_exists="replace", index=False)
    print(" Uploaded train/test splits into 'train_set' and 'test_set'.")
except Exception as e:
    print(f" Failed to upload train/test splits:", file=sys.stderr)
    print(e, file=sys.stderr)



try:
    print("\n Checking 'intents_full' (first 3 rows):")
    print(pd.read_sql("SELECT * FROM intents_full LIMIT 3;", engine))

    print("\n Checking 'train_set' (first 3 rows):")
    print(pd.read_sql("SELECT * FROM train_set LIMIT 3;", engine))

    print("\n Checking 'test_set' (first 3 rows):")
    print(pd.read_sql("SELECT * FROM test_set LIMIT 3;", engine))
except Exception as e:
    print(f" Failed to read data from database for verification:", file=sys.stderr)
    print(e, file=sys.stderr)

 Connected to PostgreSQL Database!
Generating synthetic dataset...
 Generated 1080 total samples.
 Uploaded 1080 rows into 'intents_full' table successfully!

 Train set: 864 rows |  Test set: 216 rows
 Uploaded train/test splits into 'train_set' and 'test_set'.

 Checking 'intents_full' (first 3 rows):
                                           utterance          intent
0         Any special fares or offers aavilable now?       Discounts
1                    When is flight DL607 departing?   Flight Status
2  What documents are required for medical travel...  Medical Policy

 Checking 'train_set' (first 3 rows):
                                           utterance                intent
0                HOW DOES THE CANCELLATION FEE WORK?   Cancellation Policy
1  Hey, Are sharp objects allowed in checked lugg...  Prohibited Items Faq
2            How can I apply a coupon to my booking?             Discounts

 Checking 'test_set' (first 3 rows):
                                utterance 